In [1]:
%matplotlib inline

import librosa
import soundfile as sf
import numpy as np
from hmmlearn import hmm
import matplotlib.pyplot as plt  
import os
import pickle

STRIDE_SIZE = 10.
FRAME_SIZE = 25.
N_MFCC = 16

WINDOW = 1000
WINDOW_STRIDE = 30

def find_peaks(X, Y, iterations=1):
    out = list(zip(X, Y))
    for i in range(iterations):
        temp_out = []
        for j in range(len(out)):
            if (j-1) >= 0 and (j+1) < len(out):
                if out[j-1][1] < out[j][1] and out[j][1] > out[j+1][1]:
                    temp_out.append(out[j])
        out = temp_out
    return out

def truncate_peaks(peaks, thresh = 5):
    curr = peaks[0]
    keep = []
    for p in peaks:
        if abs(p[0] - curr[0]) >= thresh:
            keep.append(curr)
        curr = p
    return keep
    
def window_number_to_sec(win_num, win_len = WINDOW):
    stamp = win_num + (win_len // 2)
    return (FRAME_SIZE + STRIDE_SIZE * (stamp - 1)) / 1000.

def get_mfcc_features(song_data, sample_rate, stride_size = STRIDE_SIZE, frame_size = FRAME_SIZE):
    mfccs = librosa.feature.mfcc(song_data, sample_rate, 
                                 n_mfcc=N_MFCC,
                                 hop_length=int(STRIDE_SIZE / 1000. * sample_rate), 
                                 n_fft=int(FRAME_SIZE / 1000. * sample_rate))
    return mfccs

def load_song(song_file):
    data, samplerate = sf.read(song_file)
    data = data[:, 0]
    mfccs = get_mfcc_features(data, samplerate)
    mfccs = np.asarray(mfccs).T
    return mfccs

def get_bpm_resp_window(song_data, sr):
    onset_env = librosa.onset.onset_strength(song_data, sr=sr)
    tempo = librosa.beat.tempo(onset_envelope=onset_env, sr=sr)
    bps = tempo / 60.0
    return 32.0 / bps
    

def continuous_prob_estimate(mfccs):
    low_score = 1e9
    low_score_index = -1

    X = []
    Y = []
    print(np.shape(mfccs))
    for i in range(0, np.shape(mfccs)[0], WINDOW_STRIDE):
        if np.shape(mfccs)[0] - WINDOW <= i:
            break
        model = hmm.GaussianHMM(n_components=1	)
        model = model.fit(mfccs[i:i+WINDOW//2], [WINDOW//2])
        curr_score = model.score(mfccs[i+WINDOW//2:i+WINDOW], [len(mfccs[i+WINDOW//2:i+WINDOW])])
        X.append(window_number_to_sec(i))
        Y.append(curr_score * -1)
        if curr_score < low_score:
            low_score = curr_score
            low_score_index = i
    return X, Y
            
def make_peak_plot(X, Y, n=3):
    peaks = find_peaks(X, Y, iterations=n)
    trunc_peaks = truncate_peaks(peaks)
    #trunc_peaks = peaks
    print(trunc_peaks)
    plt.plot(X, Y)
    # for p in peaks:
    #     plt.axvline(x=p[0], color="red")
    for p in trunc_peaks:
        plt.axvline(x=p[0], color="red")

    plt.show()

In [2]:
from pathlib import Path
home = str(Path.home())
SONG_DIR = home + "/Downloads/songdata_90/songdata/"

results = []

for filename in os.listdir(SONG_DIR):
    if ".wav" in filename:
        mfccs = load_song(SONG_DIR + filename)
        X, Y = continuous_prob_estimate(mfccs)
        peaks = find_peaks(X, Y, iterations=3)
        trunc_peaks = truncate_peaks(peaks)
        print(filename,"\n", trunc_peaks)
        results.append((filename, trunc_peaks))

result_file = open('resultsFile2', 'wb')   
# source, destination 
pickle.dump(results, result_file)                      
result_file.close() 

(20812, 16)
Galantis___Love_on_Me.wav 
 [(24.415, 35817.746999773626), (37.615, 33072.1790853665), (68.815, 38399.54971137308), (83.815, 34534.905951419685), (105.415, 42943.497938566165), (126.415, 36123.097288347526), (141.715, 32948.343342884764), (157.615, 41408.15769571066)]
(28093, 16)
Michael_Jackson___Love_Never_Felt_So_Good_(Fedde_Le_Grand_Remix).wav 
 [(25.015, 30400.97654200179), (50.215, 33471.794604426745), (78.115, 36497.85297319396), (97.615, 28670.211362042195), (107.515, 30541.49405086595), (139.315, 63656.899339629636), (172.315, 44139.33988786393), (205.015, 30091.945894392156), (222.415, 34818.45937377306), (253.015, 42396.14474728127)]
(23607, 16)
DVBBS___Tsunami.wav 
 [(80.215, 51878.42298895114), (110.215, 37598.9444336482), (126.715, 83626.8072784835), (171.115, 34357.60584192023)]
(27453, 16)
Galantis___Peanut_Butter_Jelly_(Extended_Mix).wav 
 [(46.015, 52763.95455091294), (55.315, 34834.24128516038), (80.515, 32463.34656155697), (109.015, 48868.32294265745), (

(32865, 16)
MAGIC____Rude_(Zedd_Remix).wav 
 [(34.915, 36002.921557262904), (50.215, 33706.40677554237), (64.315, 35519.708579979306), (72.115, 32736.974085532842), (118.915, 47599.59264747566), (148.915, 35146.80639685167), (163.615, 43388.30425181256), (200.215, 37765.41756072378), (251.815, 48642.373107696854), (281.815, 35677.486056479254)]
(19940, 16)
Cheat_Codes_(ft._Demi_Lovato)___No_Promises_(Ashworth_Remix).wav 
 [(82.915, 40544.82162993306), (101.515, 41808.722433710675), (151.615, 33514.4705603798)]
(16032, 16)
Deorro_(ft._Pitbull___Elvis_Crespo)___Bailar.wav 
 [(31.315, 36686.1241918987), (46.615, 41728.21274499394), (83.215, 43821.25698202089), (106.315, 36680.27480020968)]
(19008, 16)
TieÌsto___Summer_Nights_(feat._John_Legend).wav 
 [(46.915, 33429.37863434602), (81.115, 46789.91862619153)]
(23538, 16)
Calvin_Harris___I_Need_Your_Love.wav 
 [(27.415, 34619.214016388745), (43.315, 39712.32831926995), (58.615, 52051.66151198215), (76.015, 30973.670332870504), (94.315, 330

(27399, 16)
Lucas___Steve___Up_Till_Dawn_(On_The_Move)_Club_Mix.wav 
 [(38.815, 41069.93386764574), (58.915, 35407.22651687473), (98.215, 45067.64400179978), (111.715, 31653.02159497422), (133.315, 53418.95426760379), (157.915, 35566.72807333001), (170.215, 33843.25266922189), (186.415, 32263.480966960586), (212.515, 44182.79458261156)]
(18503, 16)
MOTi___Livin_4_Ya.wav 
 [(35.515, 33060.687570173206), (64.015, 41226.51147835818), (96.415, 44547.274549496506), (127.015, 32902.540119907346)]
(19080, 16)
Valentino_Khan___Deep_Down_Low.wav 
 [(64.015, 177854.05938819647), (101.815, 34747.514077008), (140.215, 159808.54899962788)]
(26160, 16)
Martin_Jensen___Solo_Dance.wav 
 [(21.715, 39282.75593517621), (50.815, 38819.137552536435), (112.315, 51220.51594536691), (145.315, 33318.42225782321), (159.415, 53880.23796637767), (176.515, 32427.657263460893), (190.615, 37713.261657637995), (208.315, 38908.91248428652)]
(27569, 16)
Martin_Garrix_(feat_Dua_Lipa)Scared_To_Be_Lonely_(Brooks_Remix).wa

In [5]:
import csv
import pandas as pd

df = pd.read_csv("songs_fixed.csv")
list.sort(results, key = lambda x : x[0])

detection_count = 0
drop_count = 0
for i in range(len(results)):
    drops = str(df.iloc[[i]]["Drops"][i]).split(", ")
    peaks = results[i][1]
    temp_detection_count = 0
    songname = df.iloc[[i]]["Song Name"][i]
    drops_found = []
    print(drops, peaks)
    for d in drops:
        for p in peaks:
            dval = float(d)
            pval = float(p[0])
            if pval >= dval - 10 and pval <= dval + 5:
                temp_detection_count += 1
                drops_found.append(d)
                break
    if temp_detection_count < len(drops):
        print(songname)
        print(drops_found)
        print(results[i][0])
    drop_count += len(drops)
    detection_count += temp_detection_count

print(detection_count / drop_count)

['68.1', '158.1'] [(27.715, 34794.97175649199), (71.215, 43239.054424007496), (83.815, 33148.31481641887), (104.215, 47437.559570603284), (162.415, 34256.742264898254), (175.615, 31281.728675335726)]
['72.4', '158.6'] [(47.215, 36231.16333406424), (75.415, 44233.258258061294), (92.215, 44740.857122935944), (122.515, 32839.34896220612)]
Alan_Walker___The_Spectre
['72.4']
Alan_Walker___The_Spectre.wav
['94.0', '172.1', '233.1'] [(36.115, 34014.5726776806), (55.015, 35257.06601089104), (94.615, 40492.6877428494), (112.615, 31997.68251042169), (121.915, 31941.22639373947), (132.415, 37761.729564985166), (146.215, 35138.79062741699), (172.615, 40374.85174032875), (190.915, 32185.250798694913), (209.515, 31307.19611529285), (236.215, 38208.925443901964), (258.715, 31570.160823853636)]
['48.2', '170.1'] [(46.015, 51274.26363321879), (70.015, 31182.05322601235), (77.815, 31111.501099325415), (92.815, 31221.848937072013), (115.015, 39659.08596188756), (129.115, 42041.01934007849), (166.915, 485

In [12]:
song = "Alan_Walker___The_Spectre.wav"

mfccs = load_song(SONG_DIR + song)
X, Y = continuous_prob_estimate(mfccs)


(20632, 16)


In [ ]:
make_peak_plot(X, Y, n=3)